# MATRIX CALCULATIONS with NUMPY
####    from Harel Wallach-15Oct2019- modified by Iaakov1-15Oct2019
####    modified Harel Wallach-17Oct2019

In [ ]:
import numpy as np
from numpy import linalg as LA
import math
import pandas as pd
import matplotlib as mpl
from IPython.display import display, HTML

from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.render()
    display_html(html_str.replace('table','table style="display:inline;padding:15px;"'),raw=True)


Paste matrices as tab-separated

In [ ]:
provider_str = """
1	0	0	0	0	0	0
0	1	0	0	0	0	0
0	0	1	0	0	0	0
0	0	0	1	0	0	0
0	0	0	0	1	0	0
0	0	0	0	0	1	0
0	0	0	0	0	0	1
"""

consumer_str = """
0	0	0	0	0	0	0
1	0	0	0	0	0	0
1	0	0	1	1	1	1
1	0	0	0	0	0	0
1	0	0	0	0	0	0
1	0	0	0	0	0	0
1	0	0	0	0	0	0
"""

Read the matrices into Numpy

In [ ]:
#provider_str = provider_str.replace('\t','\t0 ')
mat = np.fromstring(provider_str, dtype=int, sep='\t')  # Matrix mat elements from mstring
dim = int(math.sqrt(mat.size))    # Matrix Dimension dim from matrix mat
provider_mat = np.reshape(mat, (dim, dim))  # reshape Matrix mat to square

#consumer_str = consumer_str.replace('\t','\t0 ')
mat = np.fromstring(consumer_str, dtype=int, sep='\t')  # Matrix mat elements from mstring
dim = int(math.sqrt(mat.size))    # Matrix Dimension dim from matrix mat
consumer_mat = np.reshape(mat, (dim, dim))  # reshape Matrix mat to square

Combine two matrices to the wheighted Modularity Matrix

In [ ]:
provider_wheight = 2
consumer_wheight = 1
union_mat = provider_mat*provider_wheight + consumer_mat*consumer_wheight

Let's view:

In [ ]:
header = ['S{}'.format(i) for i in range(1, dim+1)]
indx = ['F{}'.format(i) for i in range(1, dim+1)]
provider_df = pd.DataFrame(provider_mat, columns= header, index = indx)
consumer_df = pd.DataFrame(consumer_mat, columns= header, index = indx)
union_df = pd.DataFrame(union_mat, columns= header, index = indx)

p=provider_df.style.background_gradient(cmap='Wistia').set_caption("PROVIDERS")
c=consumer_df.style.background_gradient(cmap='Wistia').set_caption("CONSUMERS")
u=union_df.style.background_gradient(cmap='Wistia').set_caption("UNION")
display_side_by_side(p,c,u)

In [80]:
## This is used for further iterations
#union_str = """
#2	1	1	1	1
#0	2	0	0	0
#0	0	2	0	0
#0	0	0	2	0
#0	0	0	0	2
#"""
#mat = np.fromstring(union_str, dtype=int, sep='\t')  # Matrix mat elements from mstring
#dim = int(math.sqrt(mat.size))    # Matrix Dimension dim from matrix mat
#union_mat = np.reshape(mat, (dim, dim))  # reshape Matrix mat to square

In [81]:
row_sum = np.sum(union_mat,axis = 1, dtype=int)
col_sum = np.sum(union_mat,axis = 0, dtype=int)
deg = np.diag(np.concatenate((col_sum,row_sum)))

In [82]:
adj = np.block([[np.zeros((dim,dim)),union_mat.T],	
               [union_mat, np.zeros((dim,dim))]])
laplacian = np.matrix(deg-adj, dtype=int)

In [83]:
print('==deg==\n', deg)
print('==adj==\n', adj)
print('==laplacian==\n', laplacian)

==deg==
 [[8 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 2 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 2 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 3 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 3 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 3 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 3 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 2 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 3 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 7 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 3 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 3 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 3 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 3]]
==adj==
 [[0. 0. 0. 0. 0. 0. 0. 2. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 2. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 2. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 2. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 2.]
 [2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 2. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0.

In [84]:
# Iaakov: added COMPUTATIONS
# Matrix RANK
sqmat_rank = np.linalg.matrix_rank(laplacian)
# Matrix TRACE
sqmat_trace = np.trace(laplacian)
# the EIGENV computation
v, w = LA.eig(laplacian)
res = np.vstack([v,w])

In [85]:
# Iaakov: changed PRINTING precision from 10 to 2
np.set_printoptions(linewidth=200, precision=2,formatter={'float_kind':'{:5.3f}'.format},)

# Iaakov: more pretty PRINTING
print('\n==== MATRIX CALCULATIONS with NUMPY ====')
print('\nINPUT MATRIX')
print(laplacian)
print("\n--->Matrix RANK: ", sqmat_rank)
print("-->Matrix TRACE:", sqmat_trace)
print('\n EIGENVALUES')
print(v[:None])  # Eigenvalues
print('\n EIGENVECTORS')
print(w)         # Eigenvectors
print('\n EIGENVALUES & related EIGENVECTORS')
print(res)      


==== MATRIX CALCULATIONS with NUMPY ====

INPUT MATRIX
[[ 8  0  0  0  0  0  0 -2 -1 -1 -1 -1 -1 -1]
 [ 0  2  0  0  0  0  0  0 -2  0  0  0  0  0]
 [ 0  0  2  0  0  0  0  0  0 -2  0  0  0  0]
 [ 0  0  0  3  0  0  0  0  0 -1 -2  0  0  0]
 [ 0  0  0  0  3  0  0  0  0 -1  0 -2  0  0]
 [ 0  0  0  0  0  3  0  0  0 -1  0  0 -2  0]
 [ 0  0  0  0  0  0  3  0  0 -1  0  0  0 -2]
 [-2  0  0  0  0  0  0  2  0  0  0  0  0  0]
 [-1 -2  0  0  0  0  0  0  3  0  0  0  0  0]
 [-1  0 -2 -1 -1 -1 -1  0  0  7  0  0  0  0]
 [-1  0  0 -2  0  0  0  0  0  0  3  0  0  0]
 [-1  0  0  0 -2  0  0  0  0  0  0  3  0  0]
 [-1  0  0  0  0 -2  0  0  0  0  0  0  3  0]
 [-1  0  0  0  0  0 -2  0  0  0  0  0  0  3]]

--->Matrix RANK:  13
-->Matrix TRACE: 48

 EIGENVALUES
[10.009 7.775 4.226 0.000 1.643 1.348 0.438 4.562 1.000 5.000 5.000 5.000 1.000 1.000]

 EIGENVECTORS
[[0.764 0.491 0.224 -0.267 0.119 0.200 0.000 -0.000 0.000 0.000 -0.000 0.000 0.000 0.000]
 [0.029 0.042 -0.352 -0.267 -0.068 -0.137 0.719 0.512 0.000 -0.00